In [2]:
%pip install langchain langchain-google-genai faiss-cpu sentence-transformers pandas langchain-google-genai flask_cors

  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_google_genai-2.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached langchain_core-0.3.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.18-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl.metadata (9.8 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.me

In [3]:
%pip install -U langchain-community

  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached propcache-0.3.0-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached yarl-1.18.3-cp312-cp312-win_amd64.whl.metadata (71 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------ --------------- 1.6/2.5 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.0 MB/s eta 0:00:00
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl (51 kB)
Using cached propcache-0.3.0-cp312-cp312-win_amd64.whl (44 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached yarl-1.18.3-cp312-cp312-win_amd64.whl (90 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import re
import os
import pandas as pd
import getpass
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
# from pyngrok import ngrok, conf  # Expose Colab server
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_google_genai import GoogleGenerativeAI  # Import Gemini API wrapper

In [ ]:
# conf.get_default().auth_token = getpass.getpass("Enter your ngrok auth token: ")

Enter your ngrok auth token: ··········


In [ ]:
# Initialize Flask App
app = Flask(__name__, template_folder="templates")
CORS(app)


In [ ]:
# ✅ Load the investor dataset
file_path = "/content/Combined file.xlsx - Sheet1.json"

with open(file_path, "r", encoding="utf-8") as file:
    investor_data = json.load(file)

# Convert JSON to DataFrame
df = pd.DataFrame(investor_data)

In [ ]:
df = df[["Name", "Industry", "Stage", "Cheque_range", "Countries", "Global_HQ", "Type", "Overview"]]
df.head()

,Name,Industry,Stage,Cheque_range,Countries,Global_HQ,Type,Overview
0,01 Ventures,Information Technology & Services,"Pre-seed, Idea, Prototype/MVP, Seed",$250K - $2M,"UK,Netherlands",Netherlands,VC,We invest in deep tech innovations including s...
1,1 4 All Group,"Biotech, IoT, SaaS, Consumer, Energy, Fintech,...","Pre-seed, Seed, Series+",$100K - $14M,"DRC Congo,Ethiopia,Kenya,South Africa,Brazil,C...",United Arab Emirates,Family office,We invest in all 6 economic sectors: Financial...
2,10D,Information Technology & Services,"Pre-seed, Idea, Prototype/MVP, Seed",$500K - $5M,Israel,Israel,VC,We invest in Israeli and Israeli-related excep...
3,10K Ventures,Sector Agnostic,"Pre-seed, Idea, Prototype/MVP, Seed",$100K - $200K,"Kenya,Nigeria,Brazil,Colombia,Mexico,India,Ind...",Germany,Family office,We invest in early stage startups and funds gl...
4,10x Founders,Information Technology & Services,"Pre-seed, Idea, Prototype/MVP, Seed",$100K - $3M,"Germany,Austria,Switzerland,France,UK,Poland,C...",Germany,VC,We invest in the most ambitious tech founders ...


In [ ]:
# ✅ Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert investor data into FAISS-compatible format
df["text"] = df.apply(lambda row:
    f"{row['Type']} investor, based in {row['Global_HQ']}, focused on {row['Industry']}. "
    f"Invests in {row['Stage']} stage with cheque range {row['Cheque_range']} in {row['Countries']}. "
    f"Overview: {row['Overview']}", axis=1)



special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Convert data into FAISS vector format
documents = [
    Document(page_content=row["text"], metadata=row.to_dict()) for _, row in df.iterrows()
]

# Store investor vectors in FAISS
faiss_index = FAISS.from_documents(documents, embedding_model)

print(f"✅ Stored {len(documents)} investors in FAISS vector database!")

✅ Stored 2572 investors in FAISS vector database!


In [ ]:
gemini_api_key = "GEMINI_API_KEY"
llm = GoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", google_api_key=gemini_api_key)

In [ ]:
# ✅ Define structured prompt template for Gemini
prompt_template = PromptTemplate(
    input_variables=["founder_details", "investor_details"],  # 🔹 Changed from "investor_list" to "investor_details"
    template="""
    You are an AI assistant specializing in startup investments.

    **Startup Details:**
    {founder_details}

    **Investor List:**
    {investor_details}

    Your task:
    - Assign an **Overall Match Score (0-100)** for each investor.
    - Provide a **category-wise breakdown (0-100)** for:
      - Industry Match
      - Investment Stage Match
      - Cheque Range Fit
      - Country Compatibility
      - Investor Type Match
      - Global HQ Relevance

    **Return ONLY valid JSON as a list (DO NOT include extra text):**
    ```json
    [
        {{
            "Investor": "<Investor Name>",
            "Overall Match Score": <score>,
            "Category Scores": {{
                "Industry Match": <score>,
                "Investment Stage Match": <score>,
                "Cheque Range Fit": <score>,
                "Country Compatibility": <score>,
                "Investor Type Match": <score>,
                "Global HQ Relevance": <score>
            }},
            "Reasoning": "<Short explanation>"
        }},
        ...
    ]
    ```
    """
)

In [ ]:
def extract_json(response_text):
    """Extract valid JSON from Gemini response using regex fallback"""
    try:
        return json.loads(response_text)  # Try direct JSON parsing
    except json.JSONDecodeError:
        try:
            json_match = re.search(r"\[.*\]", response_text, re.DOTALL)  # Extract JSON list
            if json_match:
                return json.loads(json_match.group())  # Parse extracted JSON
        except json.JSONDecodeError:
            pass
    return None  # Return None if JSON is invalid

In [ ]:
# ✅ Home Route (Serves the frontend)
@app.route('/')
def home():
    return render_template("index.html")

In [ ]:
# ✅ Fixed the function call by renaming `investor_list` to `investor_details`
@app.route("/match-investors", methods=["POST"])
def match_investors():
    data = request.json  # Get user input from frontend

    founder_profile = {
        "Industry": data["industry"],
        "Stage": data["stage"],
        "Funding Required": data["funding"],
        "Country": data["country"]
    }

    founder_text = f"{founder_profile['Industry']} startup at {founder_profile['Stage']} stage, needs {founder_profile['Funding Required']} in {founder_profile['Country']}"

    # ✅ Use FAISS to find top 5 similar investors
    faiss_matches = faiss_index.similarity_search(founder_text)

    # ✅ Create batch investor list
    investor_details = []  # 🔹 Changed from `investor_list` to `investor_details`
    for doc in faiss_matches:
        investor_details.append(
            f"Name: {doc.metadata['Name']}, Type: {doc.metadata['Type']}, Global HQ: {doc.metadata['Global_HQ']}, "
            f"Industry: {doc.metadata['Industry']}, Stage: {doc.metadata['Stage']}, Cheque Range: {doc.metadata['Cheque_range']}, "
            f"Countries: {doc.metadata['Countries']}, Overview: {doc.metadata['Overview']}"
        )

    # ✅ Fixed function call by using `investor_details`
    chain = LLMChain(llm=llm, prompt=prompt_template)
    response = chain.run(founder_details=founder_text, investor_details="\n".join(investor_details))

    match_data = extract_json(response)  # Ensure valid JSON

    if match_data:
        # ✅ Sort by Overall Match Score (High to Low)
        match_data.sort(key=lambda x: x["Overall Match Score"], reverse=True)
        print("📊 Batch Processed Match Results:", match_data)
        return jsonify(match_data)

    print("⚠️ Invalid JSON response")
    return jsonify({"error": "Invalid response from Gemini"}), 500

In [ ]:
# ✅ Start Flask with ngrok
if __name__ == '__main__':
    port = 5000
    # public_url = ngrok.connect(port).public_url
    # print(f"🚀 Public URL: {public_url}")
    app.run(port=port)

🚀 Public URL: https://d775-34-126-112-200.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 08:04:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 08:04:54] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Mar/2025 08:05:30] "POST /match-investors HTTP/1.1" 200 -


📊 Batch Processed Match Results: [{'Investor': 'Sofinnova Partners', 'Overall Match Score': 97, 'Category Scores': {'Industry Match': 100, 'Investment Stage Match': 80, 'Cheque Range Fit': 100, 'Country Compatibility': 100, 'Investor Type Match': 100, 'Global HQ Relevance': 100}, 'Reasoning': 'Excellent match across all categories. Strong industry, stage, cheque range ($1M-$60M), country, investor type, and HQ alignment. Sofinnova Partners appears to be a very strong potential investor.'}, {'Investor': 'Seventure - Life Sciences', 'Overall Match Score': 60, 'Category Scores': {'Industry Match': 100, 'Investment Stage Match': 80, 'Cheque Range Fit': 10, 'Country Compatibility': 100, 'Investor Type Match': 100, 'Global HQ Relevance': 100}, 'Reasoning': 'Excellent industry, country, investor type, and HQ match. Stage is acceptable (Series+). However, cheque range ($1M-$10M, up to $20M in multiple rounds) is significantly lower than the required $50M.'}, {'Investor': 'Sharpstone Capital', 